<a href="https://colab.research.google.com/github/dawidswitala/dw_matrix/blob/master/day_5_tunning_hyperpametres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install hyperopt

In [15]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score
%load_ext tensorboard

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin



The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [4]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign'


/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign


In [0]:
train=pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')
train.keys()

X_train, y_train = train['features'], train['labels']
X_train.shape, y_train.shape

X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim ==1: y_train = to_categorical(y_train)
if y_test.ndim ==1: y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes= y_train.shape[1]


In [0]:
def train_model(model,X_train,y_train, params_fit ={} ):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callbacks = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(X_train,
            y_train,
            batch_size = params_fit.get('batch_size', 128),
            epochs = params_fit.get('epochs', 5),
            verbose = params_fit.get('verbose', 1),
            validation_data = params_fit.get('validation_data', (X_train,y_train)),
            callbacks=[tensorboard_callbacks]
            
            )
  return model


In [0]:
def predict(model_trained, X_test, y_test, scoring = accuracy_score ):
 
  y_test_norm = np.argmax(y_test,axis=1)
  y_pred_prob = model_trained.predict(X_test)
  y_pred=np.argmax(y_pred_prob,axis=1)
  return scoring(y_test_norm,y_pred)



In [0]:
df=pd.read_csv('data/signnames.csv')
labels_dict = df.to_dict()['b']

In [0]:
def get_cnn_v5(input_shape,num_classes):
 return Sequential([
            Conv2D(filters = 32, kernel_size=(3,3), activation='relu', input_shape=input_shape) ,
            Conv2D(filters = 32, kernel_size=(3,3), activation='relu', padding='same') ,
            MaxPool2D(),
            Dropout(0.3),

            Conv2D(filters = 64, kernel_size=(3,3), activation='relu') ,
            Conv2D(filters = 64, kernel_size=(3,3), activation='relu', padding='same') ,
            MaxPool2D(),
            Dropout(0.3),

            Conv2D(filters = 64, kernel_size=(3,3), activation='relu') ,
            Conv2D(filters = 64, kernel_size=(3,3), activation='relu', padding='same') ,
            MaxPool2D(),
            Dropout(0.3),

            Flatten(),
            
            Dense(1024, activation='relu'),     
            Dropout(0.3),

            Dense(1024, activation='relu'),     
            Dropout(0.3),
            
            Dense(num_classes, activation='softmax'),
              ])




In [13]:
model = get_cnn_v5(input_shape,num_classes)
model_trained = train_model(model,X_train,y_train, params_fit={} )
predict(model_trained, X_test, y_test )

Epoch 1/5
272/272 [==============================] - 4s 14ms/step - loss: 2.4605 - accuracy: 0.3143 - val_loss: 1.0683 - val_accuracy: 0.6808
Epoch 2/5
272/272 [==============================] - 4s 13ms/step - loss: 0.8741 - accuracy: 0.7194 - val_loss: 0.3044 - val_accuracy: 0.9152
Epoch 3/5
272/272 [==============================] - 4s 13ms/step - loss: 0.4289 - accuracy: 0.8636 - val_loss: 0.1382 - val_accuracy: 0.9574
Epoch 4/5
272/272 [==============================] - 4s 13ms/step - loss: 0.3142 - accuracy: 0.9014 - val_loss: 0.0993 - val_accuracy: 0.9712
Epoch 5/5
272/272 [==============================] - 4s 13ms/step - loss: 0.2424 - accuracy: 0.9255 - val_loss: 0.0723 - val_accuracy: 0.9835


0.954875283446712

In [0]:
def get_model(params):
 return Sequential([
            Conv2D(filters = 32, kernel_size=(3,3), activation='relu', input_shape=input_shape) ,
            Conv2D(filters = 32, kernel_size=(3,3), activation='relu', padding='same') ,
            MaxPool2D(),
            Dropout(params['dropout_cnn_block_one']),

            Conv2D(filters = 64, kernel_size=(3,3), activation='relu') ,
            Conv2D(filters = 64, kernel_size=(3,3), activation='relu', padding='same') ,
            MaxPool2D(),
            Dropout(params['dropout_cnn_block_two']),

            Conv2D(filters = 128, kernel_size=(3,3), activation='relu') ,
            Conv2D(filters = 128, kernel_size=(3,3), activation='relu', padding='same') ,
            MaxPool2D(),
            Dropout(params['dropout_cnn_block_three']),

            Flatten(),
            
            Dense(1024, activation='relu'),     
            Dropout(params['dropout_dense_block_one']),

            Dense(1024, activation='relu'),     
            Dropout(params['dropout_dense_block_two']),

            
            Dense(num_classes, activation='softmax'),
              ])


In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(X_train,
            y_train,
            batch_size = int(params.get('batch_size', 128)),
            epochs = 5,
            verbose =0
            
            
            )
  score = model.evaluate(X_test,y_test,verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy) )
  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}




In [0]:
space = {
'batch_size': hp.quniform('batch_size', 100, 200, 20),
'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7)
}

In [36]:
best = fmin( 
    func_obj,
    space,
    tpe.suggest,
    25,
    Trials()
)

{'batch_size': 160.0, 'dropout_cnn_block_one': 0.4393880138013452, 'dropout_cnn_block_three': 0.309703213154121, 'dropout_cnn_block_two': 0.4694109939129939, 'dropout_dense_block_one': 0.6981222784070968, 'dropout_dense_block_two': 0.5265793354379993}
accuracy=0.9079365134239197
{'batch_size': 120.0, 'dropout_cnn_block_one': 0.3796812066720972, 'dropout_cnn_block_three': 0.37805610716720145, 'dropout_cnn_block_two': 0.38389650282975774, 'dropout_dense_block_one': 0.6054323163265533, 'dropout_dense_block_two': 0.5929974183416844}
accuracy=0.9272108674049377
{'batch_size': 120.0, 'dropout_cnn_block_one': 0.41890244339931915, 'dropout_cnn_block_three': 0.45063014259068984, 'dropout_cnn_block_two': 0.3249495766928228, 'dropout_dense_block_one': 0.6894444173225106, 'dropout_dense_block_two': 0.3833359165069733}
accuracy=0.8938775658607483
{'batch_size': 100.0, 'dropout_cnn_block_one': 0.36286774052889303, 'dropout_cnn_block_three': 0.37680319495721104, 'dropout_cnn_block_two': 0.30250615177